## Intro

This project analyzes daily stock market data and models their behavior using a Fourier Series to uncover and reconstruct underlying cyclical patterns.

### Why use a Fourier Series?

The premise is that any curve can be described as a sum of many different sine and cosine waves. What if we take a seemingly random curve, like stock market data, and see if we can recreate it with sine and cosine waves, and use that model to forecast future prices?

## Install Dependencies

Install dependencies quietly, so logs don’t reveal sensitive info like absolute paths or user-specific details.

In [1]:
%pip install pandas polygon-api-client matplotlib -qq

Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
from polygon import RESTClient
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from typing import Iterable, Optional, Tuple, Dict

## Get Daily Aggregates

Use Polygon.io REST API to get daily aggregated bars. Docs: https://github.com/polygon-io/client-python

In [11]:
API_KEY = "<insert API key here>"
client = RESTClient(api_key=API_KEY)

In [4]:
def list_daily_aggregates(symbol, start_date, end_date):
    aggs = []
    for a in client.list_aggs(ticker=symbol, multiplier=1, timespan="day", from_=start_date, to=end_date, limit=50000):
        aggs.append(a)
    return aggs

def get_bars_as_dataframe(aggs):
    bars = pd.DataFrame([{
        "timestamp": a.timestamp,
        "open": a.open,
        "high": a.high,
        "low": a.low,
        "close": a.close,
        "volume": a.volume,
        "vwap": a.vwap,
        "transactions": a.transactions
    } for a in aggs])

    # Convert timestamp to datetime and set as index
    bars["date"] = pd.to_datetime(bars["timestamp"], unit="ms")
    bars.set_index("date", inplace=True)
    bars.drop(columns=["timestamp"], inplace=True)

    return bars.sort_index()

## Transforming Raw Stock Market Data

Let's say $y(t)$ represents our sample stock market data, where it is a function of time $t$. We want to create $\hat{y}(t)$, trained on the data from $y(t)$, that closely resembles the original function while being modeled using a Fourier Series. This will enable us to input any time $t$ into $\hat{y}(t)$ so that we can forecast prices that exist outside of the current time domain.

$$
y(t) \;\rightarrow\; \text{sample stock market data}
$$

$$
\hat{y}(t) =
\underbrace{a + bt}_{\text{linear drift}}
+
\underbrace{\sum_{k=0}^{K-1}\big(c_k\cos(\omega_k t) + d_k\sin(\omega_k t)\big)}_{\text{oscillation}}
$$

where $a$ is the slope intercept, $b$ is the slope, $c_k$ is the coefficient for the $\cos(\omega_k t)$ component, $d_k$ is the coefficient for the $\sin(\omega_k t)$ component, $\omega_k$ is the angular frequency, and $K$ is the number of frequencies included in the model.

Note that $\hat{y}(t)$ has a linear drift component as well as an oscillation component. In the real world, stock prices can trend upwards or downwards, captured by the linear slope $b$; and at $t=0$, the stock price may start at a non-zero number, hence the reason we offset with the value $a$. We will remove this linear drift component from a piece of our data when calculating the dominant angular frequencies (an intermediate step expanded below), but it's important to recognize that linear drift is still a part of our final resulting function.

In order to create $\hat{y}(t)$, we can break it down into five steps, at a high-level:

1. **Prepare time series** - initialize sample data based on closing price

2. **Calculate dominant angular frequencies** - this basically tells us where the strongest sinuisoidal waves exist in our data

3. **Build design matrix** - creating a table that tells us how each of these waves combine at each point in time

4. **Ridge regression** - find the smoothest mix of waves that match the stock market data

5. **Calculate modeled data** - create the final output model

---

### 1. Prepare Time Series

#### A. Initialize Sample Data on Closing Price

We can use the closing price on each trading day to model our Fourier Series. By taking the logarithm of the closing price, we convert multiplicative changes into additive ones, which stabilizes variance and makes the changes comparable across different price levels. We will represent our sample data as $y(t)$, where it is a function of $t = 0, 1, 2, \dots, N-1$, and $N$ is the number of samples.

e.g.

$$
\text{Stock A: } \$5 \rightarrow \$10,\ +5\ \text{change}
$$
$$
\text{Stock B: } \$100 \rightarrow \$105,\ +5\ \text{change}
$$
$$
\text{Stock A: } \log(5) \approx 0.6990 \rightarrow \log(10) = 1,\ +0.3010\ \text{change}
$$
$$
\text{Stock B: } \log(100) = 2 \rightarrow \log(105) \approx 2.0212,\ +0.0212\ \text{change}
$$

Even though Stock A and B both changed by +5, a move from \$5→\$10 (100% increase) is very different from a move \$100→\$105 (5% increase).

In [5]:
def prepare_time_series(
    bars: pd.DataFrame,
    use_log: bool = True
) -> Tuple[np.ndarray, np.ndarray]:

    # Initilize the time series y and its time index t -> sample data: y(t)
    y = bars['close'].astype(float).to_numpy()
    y = np.log(y) if use_log else y
    t = np.arange(len(y), dtype=float)
    
    return y, t

### 2. Calculate Dominant Angular Frequencies

#### A. Detrend

Let's say that our sample data $ y(t) $ has a linear drift. We want to remove the linear drift component before calculating the top angular frequencies, because the Fourier transform will interpret even a slight linear drift as a very slow wave. Let's create a new function called $ \tilde{y}(t) $, so we don't modify the original sample data.

$$
y(t) \approx a + bt
$$

where $ a $ is the slope intercept and $ b $ is the slope

$$
\tilde{y}(t) = y(t) - (a + bt)
$$

$a + bt$ can also be represented as the matrix multiplication of $A$ and $\beta$, where

$$
A =
\begin{bmatrix}
1 & t_0 \\
1 & t_1 \\
1 & t_2 \\
\vdots & \vdots \\
1 & t_{N-1}
\end{bmatrix},
\qquad
\beta =
\begin{bmatrix}
a \\
b
\end{bmatrix}
$$

$$
a + bt = A\beta
$$

$$
\tilde{y}(t) = y(t) - A\beta
$$

We want to solve for the best $\beta$ using the least squares loss function to find the best linear fit, which we will denote $\beta^*$.

$$
\text{find } \beta \text{ such that } y \approx A\beta \rightarrow \beta^*
$$

$$
\beta^* = \arg\min_{\beta} \lVert y - A\beta \rVert^2
$$

$L(\beta)$ is the least squares loss function. We're going to solve the first derivative and second derivative of $L$ to get $\beta^*$.

$$
L(\beta) = \lVert y - A\beta \rVert^2
$$

Aside: $\lVert y - A\beta \rVert = \sqrt{(y - A\beta)^2}$  
therefore $\lVert y - A\beta \rVert^2 = (y - A\beta)^2$

$$
= (y - A\beta)^2
$$

$$
= (y - A\beta)^T (y - A\beta)
$$

$$
= y^Ty - y^TA\beta - \beta^TA^Ty + \beta^TA^TA\beta
$$

Aside: $y^TA\beta = \beta^TA^Ty$  
therefore $-y^TA\beta - \beta^TA^Ty = -2\,\beta^TA^Ty$

$$
= y^Ty - 2\,\beta^TA^Ty + \beta^TA^TA\beta
$$

Aside: $\frac{d}{d\beta}(\beta^T A^T A \beta) = (A^T A + (A^T A)^T)\beta = (A^T A + A^T A)\beta = 2A^T A\beta$

$$
\frac{dL}{d\beta} = 0 - 2A^Ty + 2A^TA\beta
$$

$$
\frac{d^2L}{d\beta^2} = 0 + 0 + 2A^TA
$$

$\frac{dL}{d\beta} = 0$ indicates a unique global minimum when $\frac{d^2L}{d\beta^2}$ (aka the Hessian) is positive definite. We just need to prove that $A^TA$ is positive definite (ignoring the coefficient 2).

A symmetric matrix like $A^TA$ is positive definite if:

$$
z^TA^TAz > 0 \text{ for all } z \neq 0
$$

where $z$ is a non-zero vector.

$$
\begin{align*}
z^TA^TAz 
&= (Az)^T(Az) \\
&= (Az)^2 \\
&= \lVert Az \rVert^2
\end{align*}
$$

The two columns of $A$ are linearly independent - not multiples of each other.

Since the columns of $A$ are linearly independent, then $Az = 0$ only has the trivial solution $z = 0$. For all non-zero $z$ vectors (where at least one component is non-zero), $Az \neq 0$ and $\lVert Az \rVert^2 > 0$. 

Therefore, $A^TA$ is positive definite, and the same is true for any scalar multiple of $A^TA$, like $\frac{d^2L}{d\beta^2}$.

Since $\frac{d^2L}{d\beta^2}$ is positive definite, $\beta^*$ is the $\beta$ where $\frac{dL}{d\beta} = 0$.

$$
0 = -2A^Ty + 2A^TA\beta^*
$$

$$
-2A^TA\beta^* = -2A^Ty
$$

$$
A^TA\beta^* = A^Ty
$$

$$
\beta^* = (A^TA)^{-1} A^Ty
$$

We can use numpy to solve for $\beta^*$, i.e.

```python
beta = numpy.linalg.solve(ATA, ATy)
```

Just for fun, we can expand the matrix multiplication for $\beta^*$, which sheds some light on what numpy is doing under the hood.

Aside: for a $2\times 2$ matrix, $\begin{bmatrix} a & b \\ c & d \end{bmatrix}^{-1} = \frac{1}{ad - bc}\begin{bmatrix} d & -b \\ -c & a \end{bmatrix}$

$$
(A^TA)^{-1}
=
\frac{1}{\,N\sum_{i=0}^{N-1} t_i^2 - \left(\sum_{i=0}^{N-1} t_i\right)^2\,}
\begin{bmatrix}
\sum_{i=0}^{N-1} t_i^2 & -\sum_{i=0}^{N-1} t_i \\
-\sum_{i=0}^{N-1} t_i & N
\end{bmatrix}
$$

$$
A^Ty
=
\begin{bmatrix}
\sum_{i=0}^{N-1} y_i \\
\sum_{i=0}^{N-1} t_i y_i
\end{bmatrix}
$$

$$
\beta^*
=
(A^TA)^{-1}A^Ty
=
\frac{1}{\,N\sum_{i=0}^{N-1} t_i^2 - \left(\sum_{i=0}^{N-1} t_i\right)^2\,}
\begin{bmatrix}
\sum_{i=0}^{N-1} t_i^2 & -\sum_{i=0}^{N-1} t_i \\
-\sum_{i=0}^{N-1} t_i & N
\end{bmatrix}
\begin{bmatrix}
\sum_{i=0}^{N-1} y_i \\
\sum_{i=0}^{N-1} t_i y_i
\end{bmatrix}
$$

$$
\beta^*
=
\frac{1}{\,N\sum_{i=0}^{N-1} t_i^2 - \left(\sum_{i=0}^{N-1} t_i\right)^2\,}
\begin{bmatrix}
\left(\sum_{i=0}^{N-1} t_i^2\right)\left(\sum_{i=0}^{N-1} y_i\right)
-
\left(\sum_{i=0}^{N-1} t_i\right)\left(\sum_{i=0}^{N-1} t_i y_i\right)
\\
-\left(\sum_{i=0}^{N-1} t_i\right)\left(\sum_{i=0}^{N-1} y_i\right)
+
N\left(\sum_{i=0}^{N-1} t_i y_i\right)
\end{bmatrix}
$$

---

#### B. Hann Window

A Hann window is a smoothing function applied to our sample data before taking its Fourier transform to reduce the sharp discontinuities at its boundaries. By softening these boundaries, we reduce spectral leakage and keep our frequency components more concentrated around their true values.

$$
h(n) = \frac{1}{2}\left(1 - \cos\left(\frac{2\pi n}{N-1}\right)\right)
\quad n = [0, N-1]
$$

$$
h(0) = h(N-1) = 0
$$

$$
\tilde{y} \;\Rightarrow\; h\,\tilde{y}
$$

---

#### C. Real Fast Fourier Transform (RFFT)

We convert the time-domain signal $\tilde{y}(t)$ into the frequency domain using the Discrete Fourier Transform (computed efficiently via the RFFT) to extract dominant angular frequencies $\omega_k$.

$$
F(k) = \sum_{n=0}^{N-1} \tilde{y}(n) \, e^{-j\, 2\pi \frac{kn}{N}}
$$

where $F(k)$ is the complex DFT coefficient at frequency bin $k$, $N$ is the number of samples, and $j$ is the imaginary unit $\sqrt{-1}$. 

Because $\tilde{y}(t)$ is real-valued, the DFT satisfies the Hermitian symmetry $F(k) = \overline{F(N-k)}$, meaning the spectrum is mirrored and only the first $\left\lfloor \tfrac{N}{2} \right\rfloor + 1$ coefficients contain unique frequency information. 

The magnitude $|F(k)|$ represents the strength of the component at angular frequency $\omega_k$.

$$
\omega_k = 2\pi \frac{k}{N}.
$$

$$
\hat{y}(t) =
a + bt +
\sum_{k=0}^{K-1}
\big(c_k \cos(\omega_k t) + d_k \sin(\omega_k t)\big)
$$

where $K = \left\lfloor \tfrac{N}{2} \right\rfloor + 1$, the number of frequencies selected from the RFFT.

In [6]:
def calculate_dominant_angular_frequencies(
    y: np.ndarray,
    t: np.ndarray,
    use_detrend: bool = True,
    use_hann: bool = True,
    sample_spacing: float = 1.0,
) -> Tuple[np.ndarray, np.ndarray]:

    # If sampled data is too small, return
    N = y.size
    if N < 2:
        return np.array([], dtype=float), np.array([], dtype=float)

    y_tilda = y.copy()

    # (A) Detrend
    if use_detrend:
        A = np.column_stack([np.ones_like(t), t])
        ATA = A.T @ A
        ATy = A.T @ y
        beta = np.linalg.solve(ATA, ATy)
        y_tilda = y - (A @ beta)

    # (B) Hann window
    if use_hann:
        y_tilda = y_tilda * np.hanning(N)

    # (C) Fast Fourier Transform (FFT)
    F = np.fft.rfft(y_tilda)                              
    freqs = np.fft.rfftfreq(N, d=sample_spacing)     
    magnitudes = np.abs(F)                                
    omegas = 2.0 * np.pi * freqs
    
    return omegas, magnitudes

### 3. Build Design Matrix

We construct the design matrix $X$ as:

$$
X =
\begin{bmatrix}
1 & t_0 & \cos(\omega_0 t_0) & \sin(\omega_0 t_0) & \cdots & \cos(\omega_{K-1} t_0) & \sin(\omega_{K-1} t_0) \\
1 & t_1 & \cos(\omega_0 t_1) & \sin(\omega_0 t_1) & \cdots & \cos(\omega_{K-1} t_1) & \sin(\omega_{K-1} t_1) \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
1 & t_{N-1} & \cos(\omega_0 t_{N-1}) & \sin(\omega_0 t_{N-1}) & \cdots & \cos(\omega_{K-1} t_{N-1}) & \sin(\omega_{K-1} t_{N-1})
\end{bmatrix}
$$


Note that the first two columns correspond to the linear drift component in $\hat{y}$, and the following columns correspond to the oscillation component. 

In the Ridge Regression step, we will use the design matrix $X$ to calculate $\theta$ such that $\hat{y} = X \theta$.

In [7]:
def build_design_matrix(
    t: np.ndarray,
    omegas: np.ndarray,
    include_linear_drift: bool = True
) -> np.ndarray:
    cols = []
    
    if include_linear_drift:
        cols.append(np.ones_like(t, dtype=float))
        cols.append(t.astype(float))
        
    for w in omegas:
        cols.append(np.cos(w * t))
        cols.append(np.sin(w * t))
    
    X = np.column_stack(cols).astype(float)
    return X

### 4. Ridge Regression

We can write our model in matrix form as $\hat{y} = X\theta$, where

$$
X =
\begin{bmatrix}
1 & t_0 & \cos(\omega_0 t_0) & \sin(\omega_0 t_0) & \cdots & \cos(\omega_{K-1} t_0) & \sin(\omega_{K-1} t_0) \\
1 & t_1 & \cos(\omega_0 t_1) & \sin(\omega_0 t_1) & \cdots & \cos(\omega_{K-1} t_1) & \sin(\omega_{K-1} t_1) \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
1 & t_{N-1} & \cos(\omega_0 t_{N-1}) & \sin(\omega_0 t_{N-1}) & \cdots & \cos(\omega_{K-1} t_{N-1}) & \sin(\omega_{K-1} t_{N-1})
\end{bmatrix},
\qquad
\theta =
\begin{bmatrix}
a \\
b \\
c_0 \\
d_0 \\
\vdots \\
c_{K-1} \\
d_{K-1}
\end{bmatrix}
$$

$$
\hat{y} =
\begin{bmatrix}
\hat{y}(t_0) \\
\hat{y}(t_1) \\
\vdots \\
\hat{y}(t_{N-1})
\end{bmatrix}
=
a + bt + \sum_{k=0}^{K-1}\big(c_k\cos(\omega_k t) + d_k\sin(\omega_k t)\big)
$$

We want to solve for the best $\theta$ using the least squares loss function with ridge regularization, which we will denote $\theta^*$.

$$
\text{find }\theta\text{ such that } y \approx X\theta \;\rightarrow\; \theta^*
$$

$$
\theta^* = \arg\min_{\theta}\left(\lVert y - X\theta\rVert^2 + \alpha\lVert\theta\rVert^2\right)
$$

where $\alpha \geq 0$.

Note that this process is very similar to how we solved for $\beta^*$ for $y \approx A\beta$, except for one key difference — the ridge penalty $\alpha\lVert\theta\rVert^2$. We introduce $\alpha\lVert\theta\rVert^2$ to make the problem well-posed when the columns of $X$ are correlated.

Recall that for a simple linear drift model,

$$
A =
\begin{bmatrix}
1 & t_0 \\
1 & t_1 \\
\vdots & \vdots \\
1 & t_{N-1}
\end{bmatrix}
$$

the two columns $1$ and $t$ are completely independent and will never be correlated.

However, for the design matrix $X$, there is a possibility that some of the oscillation components could be correlated, which would cause the coefficients in $\theta$ to become large and unstable.

The ridge penalty $\alpha\lVert\theta\rVert^2$ shrinks these coefficients towards zero, especially important for the $c_k$ and $d_k$ terms, in order to make the fit more robust.

$L(\theta)$ is the least squares loss function. We're going to solve the first derivative and second derivative of $L$ to get $\theta^*$

$$
L(\theta) = \lVert y - X\theta \rVert^2 + \alpha \lVert \theta \rVert^2
$$

Aside: $\lVert y - X\theta \rVert = \sqrt{(y - X\theta)^2}$  
therefore $\lVert y - X\theta \rVert^2 = (y - X\theta)^2$

$$
= (y - X\theta)^2 + \alpha \theta^2
$$
$$
= (y - X\theta)^T (y - X\theta) + \alpha \theta^T \theta
$$
$$
= y^T y - y^T X \theta - \theta^T X^T y + \theta^T X^T X \theta + \alpha \theta^T \theta
$$

Aside: $y^T X \theta = \theta^T X^T y$  
therefore $-y^T X \theta - \theta^T X^T y = -2 \theta^T X^T y$

$$
= y^T y - 2\theta^T X^T y + \theta^T X^T X \theta + \alpha \theta^T \theta
$$

$$
\frac{dL}{d\theta} = 0 - 2X^T y + 2X^T X \theta + 2\alpha \theta
$$

$$
\begin{aligned}
\frac{d^2L}{d\theta^2}
&= 0 + 0 + 2X^T X + 2\alpha I \\
&= 2 (X^T X + \alpha I)
\end{aligned}
$$

$\frac{dL}{d\theta} = 0$ indicates a unique global minimum when $\frac{d^2L}{d\theta^2}$ is positive definite, and a (not necessarily unique) global minimum when positive semidefinite. We just need to prove that $X^T X + \alpha I$ is positive definite or positive semidefinite (ignoring the coefficient $2$).

A symmetric matrix like $X^TX + \alpha I$ is positive definite if:

$$
z^T(X^TX + \alpha I)z > 0 \text{ for all } z \neq 0
$$

and positive semidefinite if:

$$
z^T(X^TX + \alpha I)z \geq 0 \text{ for all } z \neq 0
$$

where $z$ is a non-zero vector and $\alpha$ $\geq$ 0.

Since $\alpha \geq 0$, there are two cases to consider:

*Case 1: $\alpha = 0$*

$$
\begin{aligned}
z^T (X^T X + \alpha I) z
&= z^T X^T X z \\
&= (Xz)^T (Xz) \\
&= (Xz)^2 \\
&= \lVert Xz \rVert^2
\end{aligned}
$$

Because the columns of $X$ may be linearly dependent, there exists some non-zero $z$ vector such that $Xz = 0$ and $\lVert Xz \rVert^2 = 0$.

$$
\lVert Xz \rVert^2 \geq 0
$$

Therefore $X^T X + \alpha I$, where $\alpha = 0$, is positive semidefinite.

*Case 2: $\alpha > 0$*

$$
\begin{aligned}
z^T (X^T X + \alpha I) z
&= z^T X^T X z + \alpha z^T z \\
&= (Xz)^T (Xz) + \alpha z^T z \\
&= \lVert Xz \rVert^2 + \alpha \lVert z \rVert^2
\end{aligned}
$$

We have proven that $\lVert Xz \rVert \ge 0$ earlier, $\lVert z \rVert^2 > 0$ because $z$ is a non-zero vector, and $\alpha > 0$.

$$
\lVert Xz \rVert^2 + \alpha \lVert z \rVert^2 > 0
$$

Therefore, $X^T X + \alpha I$, where $\alpha > 0$, is positive definite.

Since $\frac{d^2L}{d\theta^2}$ is positive semidefinite when $\alpha = 0$ and positive definite when $\alpha > 0$, $\theta^*$ is the $\theta$ where $\frac{dL}{d\theta} = 0$.

$$
\frac{dL}{d\theta} = -2 X^T y + 2 X^T X \theta + 2 \alpha \theta
$$

$$
0 = -2 X^T y + 2 X^T X \theta^* + 2 \alpha \theta^*
$$
$$
2 X^T y = 2 X^T X \theta^* + 2 \alpha \theta^*
$$
$$
X^T y = X^T X \theta^* + \alpha \theta^*
$$
$$
X^T y = (X^T X + \alpha I) \theta^*
$$
$$
\theta^* = (X^T X + \alpha I)^{-1} X^T y
$$

We now have the ridge solution $\theta^*$. However, this expression can become computationally unstable when $X$ is wide, which is the case if we add lots of correlated angular frequencies to build $X$.

In order to make the solution even more robust, we can use the Singular Value Decomposition (SVD) of the design matrix, and substitute those terms into the ridge solution.

$$
X = U S V^T
$$

where $U$ is an orthonormal matrix describing the output directions in $X\theta$-space ($\hat{y}$ space), $S$ is a diagonal matrix of singular values describing how strongly each direction in $\theta$-space is scaled by $X$, and $V$ is an orthonormal matrix describing the input directions in $\theta$-space.

$$
\begin{aligned}
X^T X
&= (USV^T)^T (USV^T) \\
&= V S U^T U S V^T \\
&= V S I S V^T \\
&= V S^2 V^T
\end{aligned}
$$

Aside: $U^T U = I$

Aside: $S I S = S^2$

$$
\begin{aligned}
X^T y
&= (USV^T)^T y \\
&= V S U^T y
\end{aligned}
$$

$$
\begin{aligned}
\theta^*
&= (X^T X + \alpha I)^{-1} X^T y \\
&= (V S^2 V^T + \alpha I)^{-1} V S U^T y \\
&= V (S^2 + \alpha I)^{-1} V^T V S U^T y \\
&= V (S^2 + \alpha I)^{-1} I S U^T y \\
&= V (S^2 + \alpha I)^{-1} S U^T y \\
&= V \left( \frac{S}{S^2 + \alpha I} \right) U^T y
\end{aligned}
$$

Since $S$ is diagonal, let $s = [s_1, s_2, \ldots, s_R]^T$ represent the diagonal entries of $S$, where $R$ is the rank of $X$ (the number of non-zero singular values).

$$
\begin{aligned}
\theta^*
&= V \left( \frac{s}{s^2 + \alpha} (U^T y) \right)
\end{aligned}
$$

In [8]:
def ridge_regression(
    X: np.ndarray,
    y: np.ndarray,
    alpha: float = 0.0,
) -> np.ndarray:

    # Calculate U, s, and V^T using Singular Value Decomposition
    # where s is a vector representing the singular values of S
    U, s, VT = np.linalg.svd(X, full_matrices=False)

    # Calculate theta
    theta = VT.T @ ((s / (s**2 + alpha)) * (U.T @ y))
    
    return theta

### 5. Calculate Modeled Data

Now that we've solved for $X$ and $\theta$, we can fully calculate $\hat{y} = X\theta$.

In [9]:
def calculate_modeled_data(
    X: np.ndarray,
    theta: np.ndarray,
    use_log: bool = True,
) -> np.ndarray:

    y_hat = X @ theta
    return np.exp(y_hat) if use_log else y_hat

In [10]:
symbol = "NVDA"

bars_start_date, bars_end_date = "2025-01-01", "2025-09-30"
    
aggs = list_daily_aggregates(symbol, bars_start_date, bars_end_date)
bars = get_bars_as_dataframe(aggs)

y, t = prepare_time_series(bars)
omegas, magnitudes = calculate_dominant_angular_frequencies(y, t)
X = build_design_matrix(t, omegas)
theta = ridge_regression(X, y)
y_hat = calculate_modeled_data(X, theta)

print(bars[:-10])
print(y_hat[:-10])

                       open     high       low   close       volume      vwap  \
date                                                                            
2025-01-02 05:00:00  136.00  138.880  134.6300  138.31  198226166.0  137.1925   
2025-01-03 05:00:00  140.01  144.900  139.7300  144.47  229300628.0  143.4879   
2025-01-06 05:00:00  148.59  152.156  147.8201  149.43  265356909.0  150.2312   
2025-01-07 05:00:00  153.03  153.130  140.0100  140.14  351724174.0  143.8269   
2025-01-08 05:00:00  142.58  143.950  137.5600  140.11  227288848.0  140.6542   
...                     ...      ...       ...     ...          ...       ...   
2025-09-10 04:00:00  176.64  179.290  175.4700  177.33  226852020.0  177.4580   
2025-09-11 04:00:00  179.68  180.280  176.4800  177.17  151159274.0  177.7323   
2025-09-12 04:00:00  177.77  178.595  176.4500  177.82  124911026.0  177.7448   
2025-09-15 04:00:00  175.67  178.850  174.5100  177.75  147061559.0  176.5506   
2025-09-16 04:00:00  177.00 